In [ ]:
!pip install soundata

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.0/162.0 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.0/97.0 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.8/102.8 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.7/51.7 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.7/142.7 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 429.9/429.9 kB 29.4 MB/s eta 0:00:00


In [ ]:
import soundata

dataset = soundata.initialize('urbansound8k')
dataset.download()  # download the dataset
dataset.validate()  # validate that all the expected files are there

example_clip = dataset.choice_clip()  # choose a random example clip
print(example_clip)  # see the available data


 15%|█▍        | 836M/5.61G [17:12<1:26:02, 997kB/s] 

In [ ]:
import os
import numpy as np
import pandas as pd
import librosa
import librosa.display
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models



In [ ]:
SR = 22050
DURATION = 4.0
N_MELS = 128
N_FFT = 1024
HOP_LENGTH = 512

def load_clip_melspec(clip):
    """
    Carga un clip de UrbanSound8K y devuelve log-mel spectrogram de shape (T, N_MELS)
    """
    y = clip.audio
    sr = clip.sample_rate

    # Resample
    if sr != SR:
        y = librosa.resample(y, orig_sr=sr, target_sr=SR)

    # Duración fija
    max_len = int(SR * DURATION)
    if len(y) < max_len:
        y = np.pad(y, (0, max_len - len(y)))
    else:
        y = y[:max_len]

    # Mel-spectrogram
    S = librosa.feature.melspectrogram(
        y=y,
        sr=SR,
        n_fft=N_FFT,
        hop_length=HOP_LENGTH,
        n_mels=N_MELS
    )
    S_db = librosa.power_to_db(S, ref=np.max)  # (N_MELS, T)

    # Transponer a (T, N_MELS) para usar Conv1D sobre el tiempo
    return S_db.T


In [ ]:
def load_data_for_fold_mel(dataset, test_fold):
    X_train, y_train = [], []
    X_test, y_test = [], []

    for cid in dataset.clip_ids:
        clip = dataset.clip(cid)
        mel = load_clip_melspec(clip)      # (T, N_MELS)
        label = clip.class_id
        fold = clip.fold

        if fold == test_fold:
            X_test.append(mel)
            y_test.append(label)
        else:
            X_train.append(mel)
            y_train.append(label)

    return np.array(X_train), np.array(y_train), np.array(X_test), np.array(y_test)


In [ ]:
def prepare_cnn_lstm_features(X):
    """
    Asegura que X sea un np.array de shape (num_samples, T, N_MELS)
    """
    return np.array(X)


In [ ]:
def build_cnn_lstm_model(n_timesteps, n_mels):
    inputs = layers.Input(shape=(n_timesteps, n_mels))

    x = layers.Conv1D(64, kernel_size=5, activation='relu', padding='same')(inputs)
    x = layers.MaxPooling1D(pool_size=2)(x)

    x = layers.Conv1D(128, kernel_size=5, activation='relu', padding='same')(x)
    x = layers.MaxPooling1D(pool_size=2)(x)

    x = layers.LSTM(64)(x)
    x = layers.Dense(64, activation='relu')(x)
    outputs = layers.Dense(10, activation='softmax')(x)

    model = models.Model(inputs=inputs, outputs=outputs)

    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model


In [ ]:
accuracies_cnn_lstm = []

print("\n===================================================")
print("🔵 INICIANDO 10-FOLD CROSS VALIDATION (CNN + LSTM)")
print("===================================================\n")

for fold in range(1, 11):

    print(f"\n===================================================")
    print(f"🚀 Ejecutando Fold {fold} ...")
    print("===================================================\n")

    # 1. Cargar datos del fold con mel-spectrogram
    X_train_raw, y_train, X_test_raw, y_test = load_data_for_fold_mel(dataset, test_fold=fold)

    print(f"Fold {fold}: Train={len(X_train_raw)}, Test={len(X_test_raw)}")
    print(f"Mel shapes: train={X_train_raw.shape}, test={X_test_raw.shape}")

    # 2. Preparar tensores para CNN + LSTM (T, N_MELS)
    X_train_cnnlstm = prepare_cnn_lstm_features(X_train_raw)
    X_test_cnnlstm  = prepare_cnn_lstm_features(X_test_raw)

    n_timesteps = X_train_cnnlstm.shape[1]
    n_mels = X_train_cnnlstm.shape[2]

    print(f"Input shape para modelo: (timesteps={n_timesteps}, mels={n_mels})")

    # 3. Crear modelo
    cnn_lstm_model = build_cnn_lstm_model(n_timesteps, n_mels)

    # 4. Entrenar
    history = cnn_lstm_model.fit(
        X_train_cnnlstm, y_train,
        epochs=15,
        batch_size=32,
        validation_data=(X_test_cnnlstm, y_test),
        verbose=1
    )

    # 5. Evaluar
    test_loss, test_acc = cnn_lstm_model.evaluate(X_test_cnnlstm, y_test, verbose=0)
    accuracies_cnn_lstm.append(test_acc)

    print(f"\n🎯 Accuracy del Fold {fold}: {test_acc:.4f}")
    print("---------------------------------------------------")

# ---------------------------------------------------
# RESULTADOS FINALES
# ---------------------------------------------------
import numpy as np

print("\n===================================================")
print("🔵 RESULTADOS DEL 10-FOLD CROSS VALIDATION (CNN + LSTM)")
print("===================================================\n")

print("Accuracies por fold:", accuracies_cnn_lstm)

mean_acc = np.mean(accuracies_cnn_lstm)
std_acc  = np.std(accuracies_cnn_lstm)

print(f"\n📌 Accuracy promedio:      {mean_acc:.4f}")
print(f"📌 Desviación estándar:    {std_acc:.4f}")

print("\n===================================================")
print("🏁 ENTRENAMIENTO COMPLETO (CNN + LSTM)")
print("===================================================")
